In [2]:
import random as rd
import astropy.io.fits as apf
import batman
import eleanor
import emcee
import getpass
import glob
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
from matplotlib.backends.backend_pdf import PdfPages
from multiprocessing import Pool
from multiprocessing import Process
import multiprocessing
import numpy as np
import os
import pandas as pd
import shutil
import time as tm 
from astroquery.mast import Catalogs
import astropy.units as units
from astropy.wcs import WCS
import math
import astropy.io.fits as apf
from astropy.stats import sigma_clip
import warnings
warnings.filterwarnings("ignore")
from wotan import flatten
import lightkurve as lk
import corner
import numpy as np
import juliet
import matplotlib.gridspec as gridspec
import mpl_axes_aligner

from astropy.timeseries import BoxLeastSquares
from ldtk import LDPSetCreator, BoxcarFilter, TabulatedFilter, SVOFilter
from ldtk.filters import tess, sdss_z
from astroquery import svo_fps

import mr_forecast as mr
import numba
from math import floor
from functools import partial


from transitleastsquares import (
    transitleastsquares,
    cleaned_array,
    catalog_info,
    transit_mask
    )




In [3]:
df_mdwarfs = pd.read_csv('PS_2024.01.19_12.24.27.csv', skiprows = 103)
set(df_mdwarfs.discoverymethod)


{'Imaging', 'Radial Velocity', 'Transit', 'Transit Timing Variations'}

In [4]:
df_imaging_mdwarfs = df_mdwarfs[df_mdwarfs.discoverymethod == 'Imaging'].reset_index(drop=True)
df_RV_mdwarfs = df_mdwarfs[df_mdwarfs.discoverymethod == 'Radial Velocity'].reset_index(drop=True)
df_transit_mdwarfs = df_mdwarfs[df_mdwarfs.discoverymethod == 'Transit'].reset_index(drop=True)
df_transit_mdwarfs

,pl_name,hostname,default_flag,sy_snum,sy_pnum,discoverymethod,disc_year,disc_facility,soltype,pl_controv_flag,...,sy_vmagerr2,sy_kmag,sy_kmagerr1,sy_kmagerr2,sy_gaiamag,sy_gaiamagerr1,sy_gaiamagerr2,rowupdate,pl_pubdate,releasedate
0,TRAPPIST-1 b,TRAPPIST-1,1,1,7,Transit,2016,La Silla Observatory,Published Confirmed,0,...,-0.2000,10.296,0.023,-0.023,15.64510,0.001448,-0.001448,2022-11-14,2021-02,2022-11-14
1,TRAPPIST-1 c,TRAPPIST-1,1,1,7,Transit,2016,La Silla Observatory,Published Confirmed,0,...,-0.2000,10.296,0.023,-0.023,15.64510,0.001448,-0.001448,2022-11-14,2021-02,2022-11-14
2,TRAPPIST-1 d,TRAPPIST-1,1,1,7,Transit,2016,La Silla Observatory,Published Confirmed,0,...,-0.2000,10.296,0.023,-0.023,15.64510,0.001448,-0.001448,2022-11-14,2021-02,2022-11-14
3,TRAPPIST-1 e,TRAPPIST-1,1,1,7,Transit,2017,Multiple Observatories,Published Confirmed,0,...,-0.2000,10.296,0.023,-0.023,15.64510,0.001448,-0.001448,2022-11-14,2021-02,2022-11-14
4,TRAPPIST-1 f,TRAPPIST-1,1,1,7,Transit,2017,Multiple Observatories,Published Confirmed,0,...,-0.2000,10.296,0.023,-0.023,15.64510,0.001448,-0.001448,2022-11-14,2021-02,2022-11-14
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
361,TOI-2018 b,TOI-2018,1,1,1,Transit,2023,Transiting Exoplanet Survey Satellite (TESS),Published Confirmed,0,...,-0.0300,7.104,0.017,-0.017,9.71999,0.000271,-0.000271,2023-06-27,2023-06,2023-06-27
362,K2-49 b,K2-49,1,1,1,Transit,2016,K2,Published Confirmed,0,...,-0.0840,12.729,0.030,-0.030,15.24440,0.000571,-0.000571,2016-07-14,2016-09,2016-07-18
363,Kepler-901 b,Kepler-901,1,1,1,Transit,2016,Kepler,Published Confirmed,0,...,-0.2000,11.907,0.019,-0.019,14.68030,0.000332,-0.000332,2016-05-06,2016-05,2016-05-10
364,Kepler-1314 b,Kepler-1314,1,1,1,Transit,2016,Kepler,Published Confirmed,0,...,-0.0526,13.220,0.032,-0.032,17.12060,0.003773,-0.003773,2016-05-06,2016-05,2016-05-10


In [5]:
df_microlensing_mdwarfs = pd.read_csv('Microlensing_plaents_NEA_01232024_1238.csv', skiprows = 101)

df_microlensing_mdwarfs = df_microlensing_mdwarfs.dropna(axis=1, how='all')
df_microlensing_mdwarfs = df_microlensing_mdwarfs[df_microlensing_mdwarfs.pl_controv_flag<1].reset_index(drop=True)
df_microlensing_mdwarfs

,pl_name,hostname,default_flag,sy_snum,sy_pnum,discoverymethod,disc_year,disc_facility,soltype,pl_controv_flag,...,sy_disterr2,sy_vmag,sy_vmagerr1,sy_vmagerr2,sy_kmag,sy_kmagerr1,sy_kmagerr2,rowupdate,pl_pubdate,releasedate
0,KMT-2018-BLG-0087L b,KMT-2018-BLG-0087L,1,1,1,Microlensing,2022,KMTNet,Published Confirmed,0,...,-1150.0,NaN,NaN,NaN,NaN,NaN,NaN,2023-04-10,2022-12,2023-04-10
1,KMT-2018-BLG-1990L b,KMT-2018-BLG-1990L,0,1,1,Microlensing,2019,KMTNet,Published Confirmed,0,...,-243.0,NaN,NaN,NaN,NaN,NaN,NaN,2019-12-05,2019-10,2019-12-05
2,MOA-2012-BLG-505L b,MOA-2012-BLG-505L,0,1,1,Microlensing,2017,MOA,Published Confirmed,0,...,-1110.0,NaN,NaN,NaN,23.93,4.13,-1.82,2019-12-05,2017-07,2019-12-05
3,MOA-2012-BLG-505L b,MOA-2012-BLG-505L,1,1,1,Microlensing,2017,MOA,Published Confirmed,0,...,-1110.0,NaN,NaN,NaN,23.93,4.13,-1.82,2019-12-05,2017-07,2017-06-22
4,MOA-bin-29 b,MOA-bin-29,0,1,1,Microlensing,2019,MOA,Published Confirmed,0,...,-1310.0,32.85,11.0,-2.95,23.66,8.07,-1.92,2019-12-05,2019-12,2019-12-05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
370,MOA-2019-BLG-008L b,MOA-2019-BLG-008L,0,1,1,Microlensing,2022,MOA,Published Confirmed,0,...,-900.0,NaN,NaN,NaN,NaN,NaN,NaN,2022-11-29,2022-09,2022-11-29
371,MOA-2019-BLG-008L b,MOA-2019-BLG-008L,0,1,1,Microlensing,2022,MOA,Published Confirmed,0,...,-900.0,NaN,NaN,NaN,NaN,NaN,NaN,2022-11-29,2022-09,2022-11-29
372,OGLE-2018-BLG-0799L b,OGLE-2018-BLG-0799L,0,1,1,Microlensing,2022,OGLE,Published Confirmed,0,...,-540.0,NaN,NaN,NaN,NaN,NaN,NaN,2023-03-15,2022-08,2023-03-15
373,OGLE-2019-BLG-1053L b,OGLE-2019-BLG-1053L,0,1,1,Microlensing,2021,KMTNet,Published Confirmed,0,...,-1000.0,NaN,NaN,NaN,NaN,NaN,NaN,2023-02-17,2021-10,2023-02-17


In [6]:
df_microlensing_mdwarfs.pl_bmassj

0       0.23000
1       0.56500
2       0.02110
3       0.02110
4       0.63000
         ...   
370    12.00000
371    12.00000
372     1.65000
373     0.00774
374     2.50000
Name: pl_bmassj, Length: 375, dtype: float64

In [7]:
def microlensing_a_to_P(df):
    P = np.sqrt(df.pl_orbsmax**3/(df.st_mass+(df.pl_bmassj/1047.57)))
    
    base = P*(df.st_mass+df.pl_bmassj/1047.57)/2/df.pl_orbsmax**3
    
    dP_up = base*np.sqrt(((3*df.pl_orbsmaxerr1/df.pl_orbsmax)**2)+
                       (((df.st_masserr1**2)+(df.pl_bmassjerr1/1047.57)**2)/(df.st_mass+df.pl_bmassj/1047.57)**2))

    dP_dwn = base*np.sqrt(((3*df.pl_orbsmaxerr2/df.pl_orbsmax)**2)+
                       (((df.st_masserr2**2)+(df.pl_bmassjerr2/1047.57)**2)/(df.st_mass+df.pl_bmassj/1047.57)**2))
    
    return np.array(P*365), np.array(dP_up*365), np.array(dP_dwn*365)





In [8]:
df_new = pd.DataFrame({'st_mass':[1.], 'pl_orbsmax':[5.2], 'pl_bmassj':[1],
         'st_masserr1':[0.01], 'st_masserr2':[0.02], 'pl_orbsmaxerr1':[0.01],
         'pl_orbsmaxerr2':[0.05], 'pl_bmassjerr2':[0.01], 
          'pl_bmassjerr1':[0.05] })

In [9]:
df_microlensing_mdwarfs = df_microlensing_mdwarfs[df_microlensing_mdwarfs['pl_orbsmax'].notna()]


In [10]:
values = microlensing_a_to_P(df_microlensing_mdwarfs)
average = [(values[1][i]+values[2][i])/2 for i in range(len(values[0]))]
df_microlensing_mdwarfs['pl_orbper'] = values[0]
df_microlensing_mdwarfs['pl_orbpererr1']=values[1]
df_microlensing_mdwarfs['pl_orbpererr2']=values[2]

df_microlensing_mdwarfs['pl_orbpererr']=average



In [11]:
df_microlensing_mdwarfs.reset_index(drop=True)

,pl_name,hostname,default_flag,sy_snum,sy_pnum,discoverymethod,disc_year,disc_facility,soltype,pl_controv_flag,...,sy_vmag,sy_vmagerr1,sy_vmagerr2,sy_kmag,sy_kmagerr1,sy_kmagerr2,rowupdate,pl_pubdate,releasedate,pl_orbpererr
0,KMT-2018-BLG-0087L b,KMT-2018-BLG-0087L,1,1,1,Microlensing,2022,KMTNet,Published Confirmed,0,...,NaN,NaN,NaN,NaN,NaN,NaN,2023-04-10,2022-12,2023-04-10,93.257254
1,KMT-2018-BLG-1990L b,KMT-2018-BLG-1990L,0,1,1,Microlensing,2019,KMTNet,Published Confirmed,0,...,NaN,NaN,NaN,NaN,NaN,NaN,2019-12-05,2019-10,2019-12-05,74.774066
2,MOA-2012-BLG-505L b,MOA-2012-BLG-505L,0,1,1,Microlensing,2017,MOA,Published Confirmed,0,...,NaN,NaN,NaN,23.93,4.13,-1.82,2019-12-05,2017-07,2019-12-05,90.038890
3,MOA-2012-BLG-505L b,MOA-2012-BLG-505L,1,1,1,Microlensing,2017,MOA,Published Confirmed,0,...,NaN,NaN,NaN,23.93,4.13,-1.82,2019-12-05,2017-07,2017-06-22,90.038890
4,MOA-bin-29 b,MOA-bin-29,0,1,1,Microlensing,2019,MOA,Published Confirmed,0,...,32.85,11.0,-2.95,23.66,8.07,-1.92,2019-12-05,2019-12,2019-12-05,44.614687
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
360,TCP J05074264+2447555 b,TCP J05074264+2447555,1,1,1,Microlensing,2018,Multiple Observatories,Published Confirmed,0,...,NaN,NaN,NaN,NaN,NaN,NaN,2021-10-19,2019-11,2021-10-19,44.869808
361,KMT-2016-BLG-1836L b,KMT-2016-BLG-1836L,0,1,1,Microlensing,2019,KMTNet,Published Confirmed,0,...,NaN,NaN,NaN,NaN,NaN,NaN,2020-04-02,2020-03,2020-04-02,32.406353
362,OGLE-2018-BLG-0799L b,OGLE-2018-BLG-0799L,0,1,1,Microlensing,2022,OGLE,Published Confirmed,0,...,NaN,NaN,NaN,NaN,NaN,NaN,2023-03-15,2022-08,2023-03-15,83.313971
363,OGLE-2019-BLG-1053L b,OGLE-2019-BLG-1053L,0,1,1,Microlensing,2021,KMTNet,Published Confirmed,0,...,NaN,NaN,NaN,NaN,NaN,NaN,2023-02-17,2021-10,2023-02-17,14.091993


In [12]:
df_microlensing_mdwarfs.pl_bmassj[302]

2.2

In [49]:
np.log10(15)

1.1760912590556813

In [50]:
R_vals = np.logspace(0, np.log10(15), 100)
R_vals

array([ 1.        ,  1.0277316 ,  1.05623224,  1.08552325,  1.11562654,
        1.14656465,  1.17836072,  1.21103855,  1.24462259,  1.27913796,
        1.3146105 ,  1.35106675,  1.38853399,  1.42704026,  1.46661437,
        1.50728593,  1.54908538,  1.59204399,  1.63619392,  1.68156819,
        1.72820077,  1.77612654,  1.82538137,  1.87600211,  1.92802665,
        1.98149391,  2.03644391,  2.09291775,  2.15095771,  2.2106072 ,
        2.27191088,  2.3349146 ,  2.39966551,  2.46621207,  2.53460408,
        2.6048927 ,  2.67713054,  2.75137165,  2.82767159,  2.90608744,
        2.98667789,  3.06950325,  3.15462548,  3.24210829,  3.33201714,
        3.4244193 ,  3.51938392,  3.61698207,  3.71728676,  3.82037307,
        3.92631812,  4.0352012 ,  4.14710378,  4.2621096 ,  4.38030471,
        4.50177757,  4.62661906,  4.7549226 ,  4.88678421,  5.02230255,
        5.16157903,  5.30471787,  5.45182618,  5.60301403,  5.75839457,
        5.91808406,  6.082202  ,  6.25087118,  6.42421783,  6.60

In [51]:
Mmedian =[]
Mplus = []
Mminus = []

for i in range(len(R_vals)):
    mass_vals =mr.Rstat2M(mean=float(R_vals[i]), std = float(np.median(np.diff(R_vals))))
    try:
        Mmedian.append(mass_vals[0])
        Mplus.append(mass_vals[1])
        Mminus.append(mass_vals[2])
    except Exception:
        print(Exception)
        print('bad index', i, R_vals[i])
        Mmedian.append(np.nan)
        Mplus.append(np.nan)
        Mminus.append(np.nan)


In [32]:
len(Mplus)

50

In [52]:
Mmedian


pd.DataFrame({'Radius': R_vals, "Mass": Mmedian, 'Mplus': Mplus, 'Mminus': Mminus}).to_csv('mass_and_radii_for_SNR_log.csv', index=False)



In [ ]:
col = df_microlensing_mdwarfs.loc[: , "pl_bmassjerr1":"pl_bmassjerr2"]
df_microlensing_mdwarfs['pl_bmassjerr'] = col.mean(axis=1)


Rmedian =[]
Rplus = []
Rminus = []

for i in range(len(df_microlensing_mdwarfs)):
    print(i)
    try:
        rad_vals =mr.Mstat2R(mean=np.array(df_microlensing_mdwarfs.pl_bmassj)[i], std=np.abs(np.array(df_microlensing_mdwarfs.pl_bmassjerr)[i]), unit='Jupiter')
        Rmedian.append(rad_vals[0])
        Rplus.append(rad_vals[1])
        Rminus.append(rad_vals[2])
    except:
        print('bad index', i)
        Rmedian.append(np.nan)
        Rplus.append(np.nan)
        Rminus.append(np.nan)

df_microlensing_mdwarfs['pl_rade'] = np.array(Rmedian)*11.2089
df_microlensing_mdwarfs['pl_radeerr1'] = np.array(Rplus)*11.2089
df_microlensing_mdwarfs['pl_radeerr2'] = np.array(Rminus)*11.2089
# Rmedian, Rplus, Rminus

In [ ]:
Rmedian

In [ ]:
df_microlensing_mdwarfs['pl_rade'] = np.array(Rmedian)*11.2089
df_microlensing_mdwarfs['pl_radeerr1'] = np.array(Rplus)*11.2089
df_microlensing_mdwarfs['pl_radeerr2'] = np.array(Rminus)*11.2089

In [ ]:
df_transit_mdwarfs = df_transit_mdwarfs[df_transit_mdwarfs['pl_orbper'].notna()]
df_transit_mdwarfs = df_transit_mdwarfs[df_transit_mdwarfs['pl_rade'].notna()]



In [ ]:
np.unique(df_planets.discoverymethod)

In [ ]:
df_toi = pd.read_csv('TOI_2024.01.04_08.22.24.csv', skiprows = 75)
df_toi = df_toi[df_toi['tfopwg_disp'].isin(['PC', 'CP'])].reset_index(drop=True)


In [ ]:
pd.set_option('display.max_columns', None)
df_toi['multi_system']=df_toi.duplicated('tid', keep=False)
df_toi = df_toi[df_toi['pl_orbper'].notna()]
df_toi = df_toi[df_toi['pl_rade'].notna()]

df_toi

In [ ]:
df_planets['multi_system'] = df_planets.duplicated('hostname', keep=False)
df_planets = df_planets[df_planets['pl_orbper'].notna()]
df_planets = df_planets[df_planets['pl_rade'].notna()]


In [ ]:
# df_toi[df_toi.tid == 103605399]['pl_orbper'] = df_toi[df_toi.tid == 103605399].pl_orbper/10
df_toi.at[155, 'pl_orbper']  =  df_toi[df_toi.tid == 103605399].pl_orbper/10

In [ ]:
df_toi[df_toi.tid == 103605399].pl_orbper

In [ ]:
df_toi = df_toi[df_toi.pl_orbper<200].reset_index(drop=True)
new_series = df_toi.toi.astype('str')

new_series = new_series.map(lambda row:row.split(".")[0])

df_toi['hostname'] = 'TOI-' + new_series.astype(str)


In [ ]:
df_toi = df_toi[~df_toi.hostname.isin(df_planets.hostname)].reset_index(drop=True)
df_toi

In [ ]:
len(df_planets[df_planets.pl_orbper>40].reset_index(drop=True))

In [ ]:
len(df_toi[df_toi.pl_orbper>40].reset_index(drop=True))

In [ ]:
len(df_planets[df_planets.multi_system == True].reset_index(drop=True))/len(df_planets.reset_index(drop=True))





In [ ]:
df_planets[df_planets.pl_orbper>20][df_planets.disc_facility == 'Transiting Exoplanet Survey Satellite (TESS)'].hostname

In [ ]:
len(df_planets[df_planets.pl_orbper>20][df_planets.disc_facility != 'Transiting Exoplanet Survey Satellite (TESS)'].reset_index(drop=True))

In [ ]:
df_toi.columns
df_toi_multis = df_toi[df_toi.multi_system == True]


In [ ]:
df_planets_multis = df_planets[df_planets.multi_system == True]


In [ ]:
%matplotlib inline

In [ ]:
import matplotlib.ticker as mticker

In [ ]:
Rmedian

In [ ]:
max(df_microlensing_mdwarfs.pl_orbper)

In [ ]:
col = df_RV_mdwarfs.loc[: , "pl_bmassjerr1":"pl_bmassjerr2"]
df_RV_mdwarfs['pl_bmassjerr'] = col.mean(axis=1)


Rmedian =[]
Rplus = []
Rminus = []

for i in range(len(df_RV_mdwarfs)):
    print(i)
    try:
        rad_vals =mr.Mstat2R(mean=np.array(df_RV_mdwarfs.pl_bmassj)[i], std=np.abs(np.array(df_RV_mdwarfs.pl_bmassjerr)[i]), unit='Jupiter')
        Rmedian.append(rad_vals[0])
        Rplus.append(rad_vals[1])
        Rminus.append(rad_vals[2])
    except:
        print('bad index', i)
        Rmedian.append(np.nan)
        Rplus.append(np.nan)
        Rminus.append(np.nan)

df_RV_mdwarfs['pl_rade'] = np.array(Rmedian)*11.2089
df_RV_mdwarfs['pl_radeerr1'] = np.array(Rplus)*11.2089
df_RV_mdwarfs['pl_radeerr2'] = np.array(Rminus)*11.2089


In [ ]:
fig = plt.figure(figsize = (46, 23), edgecolor=None)

ax = fig.add_subplot(111, xscale = 'log', yscale = 'log')

ax.patch.set_facecolor('#1F4C68') # instead of fig.patch.set_facecolor
ax.patch.set_alpha(0.65)
fig.patch.set_facecolor('None') # instead of fig.patch.set_facecolor





markers_rv, caps_rv, bars_rv = ax.errorbar(x = df_RV_mdwarfs.pl_orbper, y=df_RV_mdwarfs.pl_rade,
            xerr = [list(abs(df_RV_mdwarfs.pl_orbpererr2)), list(df_RV_mdwarfs.pl_orbpererr1)],
            yerr = [list(abs(df_RV_mdwarfs.pl_radeerr2)), list(df_RV_mdwarfs.pl_radeerr1)],
            fmt = '.', markersize = 30, ecolor = '#FFCC99', color = '#DBFEF5', zorder = 15,
                                        label = 'Radial Velocity',  linewidth = 3
           )

# markers_image, caps_image, bars_image = ax.errorbar(x = df_imaging_mdwarfs.pl_orbper, y=df_imaging_mdwarfs.pl_rade,
#             xerr = [list(abs(df_imaging_mdwarfs.pl_orbpererr2)), list(df_imaging_mdwarfs.pl_orbpererr1)],
#             yerr = [list(abs(df_imaging_mdwarfs.pl_radeerr2)), list(df_imaging_mdwarfs.pl_radeerr1)],
#             fmt = '.', markersize = 30, ecolor = '#FFCC99', color = '#0099CC', zorder = 10,
#                                         label = 'Imaging',  linewidth = 3
#            )

markers_trans, caps_trans, bars_trans = ax.errorbar(x = df_transit_mdwarfs.pl_orbper, y=df_transit_mdwarfs.pl_rade,
            xerr = [list(abs(df_transit_mdwarfs.pl_orbpererr2)), list(df_transit_mdwarfs.pl_orbpererr1)],
            yerr = [list(abs(df_transit_mdwarfs.pl_radeerr2)), list(df_transit_mdwarfs.pl_radeerr1)],
            fmt = '.', markersize = 30, ecolor = '#FFCC99', color = '#7E6043', zorder = 10,
                                        label = 'Transit',  linewidth = 3
           )


markers_toi, caps_toi, bars_toi = ax.errorbar(x = df_toi.pl_orbper, y=df_toi.pl_rade,
            xerr = [list(abs(df_toi.pl_orbpererr2)), list(df_toi.pl_orbpererr1)],
            yerr = [list(abs(df_toi.pl_radeerr2)), list(df_toi.pl_radeerr1)], zorder = 10,
            fmt = '.', markersize = 30, ecolor = '#FFCC99', color = '#CC9966', label = 'TOIs', linewidth = 3, 
           )

markers_mu, caps_mu, bars_mu = ax.errorbar(x = df_microlensing_mdwarfs.pl_orbper, y=df_microlensing_mdwarfs.pl_rade,
            xerr = [list(abs(df_microlensing_mdwarfs.pl_orbpererr2)), list(df_microlensing_mdwarfs.pl_orbpererr1)],
            yerr = [list(abs(df_microlensing_mdwarfs.pl_radeerr2)), list(df_microlensing_mdwarfs.pl_radeerr1)],
            fmt = '.', markersize = 30, ecolor = '#FFCC99', color = '#006699', zorder = 10,
                                        label = 'Microlensing',  linewidth = 3
           )

# loop through bars and caps and set the alpha value

# ax.set_xlim(0, max(df_microlensing_mdwarfs.pl_orbper)+20000)
# ax.set_ylim(0, 40)

# ax.scatter(x = df_planets_multis.pl_orbper, y=df_planets_multis.pl_rade, s = 450, color = '#CCFFFF',
#            zorder = 1, label = 'Multi-planet system'),


# ax.scatter(x = df_toi_multis.pl_orbper, y=df_toi_multis.pl_rade, s = 450, color = '#CCFFFF',
#            zorder = 1,  ),



ax.tick_params(labelsize = 45)
ax.xaxis.set_major_formatter(mticker.FormatStrFormatter('%d'))
ax.yaxis.set_major_formatter(mticker.FormatStrFormatter('%d'))

ax.tick_params(color = '#CCFFFF', which='both', width = 3)
ax.tick_params(which='major', length = 14)
ax.tick_params(which='minor', length = 8)



leg = ax.legend(fontsize = 45, fancybox = True, labelcolor = '#CCFFFF', framealpha = 0.)
# leg.get_frame().set_edgecolor('#CCFFFF')
# leg.get_frame().set_linewidth(10)
[bar.set_alpha(0.25) for bar in bars_mu]
[bar.set_alpha(0.25) for bar in bars_toi]
[bar.set_alpha(0.25) for bar in bars_trans]
[bar.set_alpha(0.25) for bar in bars_rv]
[bar.set_alpha(0.25) for bar in bars_image]

ax.tick_params(color='#CCFFFF', labelcolor='#CCFFFF')
# for spine in ax.spines.values():
#     spine.set_edgecolor('#CCFFFF')
#     spine.set_linewidth(5)

for spine in ['top', 'right', 'bottom', 'left']:
    ax.spines[spine].set_visible(False)
    
fig.savefig('demographics_log2.png', facecolor = fig.get_facecolor(), edgecolor = 'none')



In [ ]:
fig = plt.figure(figsize = (24.5, 23), edgecolor=None)

ax = fig.add_subplot(111, xscale = 'log', yscale = 'log')

ax.patch.set_facecolor('#1F4C68') # instead of fig.patch.set_facecolor
ax.patch.set_alpha(0.65)
fig.patch.set_facecolor('None') # instead of fig.patch.set_facecolor





markers_mu, caps_mu, bars_mu = ax.errorbar(x = df_microlensing_mdwarfs.pl_orbper, y=df_microlensing_mdwarfs.pl_rade,
            xerr = [list(abs(df_microlensing_mdwarfs.pl_orbpererr2)), list(df_microlensing_mdwarfs.pl_orbpererr1)],
            yerr = [list(abs(df_microlensing_mdwarfs.pl_radeerr2)), list(df_microlensing_mdwarfs.pl_radeerr1)],
            fmt = '.', markersize = 30, ecolor = 'None', color = '#006699', zorder = 10,
                                          linewidth = 3, alpha = 0., 
           )


markers_rv, caps_rv, bars_rv = ax.errorbar(x = df_RV_mdwarfs.pl_orbper, y=df_RV_mdwarfs.pl_rade,
            xerr = [list(abs(df_RV_mdwarfs.pl_orbpererr2)), list(df_RV_mdwarfs.pl_orbpererr1)],
            yerr = [list(abs(df_RV_mdwarfs.pl_radeerr2)), list(df_RV_mdwarfs.pl_radeerr1)],
            fmt = '.', markersize = 30, ecolor = '#FFCC99', color = '#DBFEF5', zorder = 15,
                                        label = 'Radial Velocity',  linewidth = 3
           )

# markers_image, caps_image, bars_image = ax.errorbar(x = df_imaging_mdwarfs.pl_orbper, y=df_imaging_mdwarfs.pl_rade,
#             xerr = [list(abs(df_imaging_mdwarfs.pl_orbpererr2)), list(df_imaging_mdwarfs.pl_orbpererr1)],
#             yerr = [list(abs(df_imaging_mdwarfs.pl_radeerr2)), list(df_imaging_mdwarfs.pl_radeerr1)],
#             fmt = '.', markersize = 30, ecolor = '#FFCC99', color = '#0099CC', zorder = 10,
#                                         label = 'Imaging',  linewidth = 3
#            )

markers_trans, caps_trans, bars_trans = ax.errorbar(x = df_transit_mdwarfs.pl_orbper, y=df_transit_mdwarfs.pl_rade,
            xerr = [list(abs(df_transit_mdwarfs.pl_orbpererr2)), list(df_transit_mdwarfs.pl_orbpererr1)],
            yerr = [list(abs(df_transit_mdwarfs.pl_radeerr2)), list(df_transit_mdwarfs.pl_radeerr1)],
            fmt = '.', markersize = 30, ecolor = '#FFCC99', color = '#7E6043', zorder = 10,
                                        label = 'Transit',  linewidth = 3
           )


markers_toi, caps_toi, bars_toi = ax.errorbar(x = df_toi.pl_orbper, y=df_toi.pl_rade,
            xerr = [list(abs(df_toi.pl_orbpererr2)), list(df_toi.pl_orbpererr1)],
            yerr = [list(abs(df_toi.pl_radeerr2)), list(df_toi.pl_radeerr1)], zorder = 10,
            fmt = '.', markersize = 30, ecolor = '#FFCC99', color = '#CC9966', label = 'TOIs', linewidth = 3, 
           )

# loop through bars and caps and set the alpha value

# ax.set_ylim(0, 40)

# ax.scatter(x = df_planets_multis.pl_orbper, y=df_planets_multis.pl_rade, s = 450, color = '#CCFFFF',
#            zorder = 1, label = 'Multi-planet system'),


# ax.scatter(x = df_toi_multis.pl_orbper, y=df_toi_multis.pl_rade, s = 450, color = '#CCFFFF',
#            zorder = 1,  ),



ax.tick_params(labelsize = 45)
ax.xaxis.set_major_formatter(mticker.FormatStrFormatter('%d'))
ax.yaxis.set_major_formatter(mticker.FormatStrFormatter('%d'))

ax.tick_params(color = '#CCFFFF', which='both', width = 3)
ax.tick_params(which='major', length = 14)
ax.tick_params(which='minor', length = 8)



leg = ax.legend(fontsize = 45, fancybox = True, labelcolor = '#CCFFFF', framealpha = 0.)
# leg.get_frame().set_edgecolor('#CCFFFF')
# leg.get_frame().set_linewidth(10)
[bar.set_alpha(0.25) for bar in bars_mu]
[bar.set_alpha(0.25) for bar in bars_toi]
[bar.set_alpha(0.25) for bar in bars_trans]
[bar.set_alpha(0.25) for bar in bars_rv]
[bar.set_alpha(0.25) for bar in bars_image]


ax.tick_params(color='#CCFFFF', labelcolor='#CCFFFF')
# for spine in ax.spines.values():
#     spine.set_edgecolor('#CCFFFF')
#     spine.set_linewidth(5)

for spine in ['top', 'right', 'bottom', 'left']:
    ax.spines[spine].set_visible(False)
ax.set_xlim(0, max(df_transit_mdwarfs.pl_orbper)+50)
    
fig.savefig('demographics_log.png', facecolor = fig.get_facecolor(), edgecolor = 'none')



In [42]:
np.diff(R_vals)[0]

0.2857142857142856

In [46]:
10//3

3